In [1]:
import re
import sys
import string
import math
from textblob import Word
from textblob import TextBlob
from collections import defaultdict

In [21]:
import collections

In [2]:
userProp = ["username","text","tweetid"]
postings = defaultdict(dict)
visit = defaultdict(dict)
Querylist = []
Length = defaultdict(dict)
docN = 0
TOPK = 10
# Score = defaultdict(dict)

In [40]:
def get_postings():
    global postings
    global docN
    f = open(r"/home/liks/Documents/实验/IR/Information-retrieval-and-data-mining-master/work3Inverted index and Boolean Retrieval Model/data_tweets/tweets.txt")  
    lines = f.readlines()#readlines()
    i = 0
    for line in lines:
        i += 1
        line = tokenize_tweet(line)
        tweetid = line[0]
        line.pop(0)
        Length[tweetid]=len(line)
       # termlist 是一个（词项：词频）的 列表
        termlist = [(k,v) for k,v in collections.Counter(line).items()]
        for te in termlist:
            if te[0] in postings.keys():
                postings[te[0]].append((tweetid,te[1]))
            else:
                postings[te[0]] = [(tweetid,te[1])]
    docN = i
    # 这样出来的posting就是一个这样的东西（字典） posting[] :
    #                                         |
    #                                        |->term1:[(tweetid,fq),(tweetid,fq)...]
    #                                       |->term2:[(tweetid,fq),(tweetid,fq)...]
    #                                       ->term3:[(tweetid,fq)...]

In [7]:
def idf(term):
    global postings
    global docN
    if term in postings.keys():
        return math.log(docN/len(postings[term]))
    else:
        return 0

In [8]:
def tfwght_query(term):
    global Querylist
#     [(k,v) for k,v in collections.Counter(a).items()]
    return (1+math.log(collections.Counter(Querylist)[term]))

In [45]:
def Search_document(documentID):
#     锁定一个document，计算这个document与当前query的打分
#     这个函数里，在get一下query的tf 直接都在这里算出来 因为tf_raw在这里
    global Querylist
    global Length
    tf_raw = []
    Score = 0
    for i in range(len(Querylist)):
        tfx = checkin(Querylist[i],documentID)
        tf_raw.append(tfx)
    for t in tf_raw:
        if t == 0:
            tf_wght = 0
        else:
            tf_wght = (1+math.log(t))
    NORMALIZE = math.sqrt(sum(w**2 for w in tf_raw))
    term_normalized = [ num/NORMALIZE for num in tf_raw]
    for i in range(len(Querylist)):
        Score += tfwght_query(Querylist[i])*idf(Querylist[i])*term_normalized[i]
    Score = Score/Length[documentID]
    return Score

In [12]:
def checkin(term,documentID):
#     计算document中是否包含term，如果包含 返回包含多少 没有就返回0
    global postings
    if term not in postings.keys():
        return 0
#     elif documentID in [te[1] for te in postings[term] ]
    else:
        for docid in postings[term]:
            if docid[0]==documentID:
                return docid[1]
    return 0        

In [13]:
class Document:
#     posting 是badge中保存每一个文档id所用到的类
    def __init__(self):
        self.documentid = 0
        self.tf = 0

In [14]:
class badge:
#     这是postings中的一个值
#     postings -> term:badge{df,postings{documentID,tf}} （df Query里要用到）
    def __init__(self):
        self.df = 0
        self.Document = None

In [15]:
def tokenize_tweet(string):
#     返回一个 列表 ，包括 username、tweetcontent和tweetid     最后的结果中不包含这三个字段
#     列表第一个值是tweetid 然后是content     [tweetid,content]
    string=string.lower()
    a = string.index("username")
    b = string.index("clusterno")
    c = string.rindex("tweetid")-1
    d = string.rindex("errorcode")
    e = string.index("text")
    f = string.index("timestr")-3
    #提取用户名、tweet内容和tweetid三部分主要信息
    string = string[c:d]+string[a:b]+string[e:f]
    # print(string)
    terms=TextBlob(string).words.singularize()
    # print(terms)

    result=[]
    for word in terms:
        expected_str = Word(word)
        expected_str = expected_str.lemmatize("v")
        if expected_str not in userProp:
            result.append(expected_str)
    return result

In [16]:
def token(query):
#     返回的就是Querylist
    query = query.lower()
    terms=TextBlob(query).words.singularize()

    result=[]
    for word in terms:#query和term做同样的处理
        expected_str = Word(word)
        expected_str = expected_str.lemmatize("v")
        result.append(expected_str)
    return result

In [55]:
def Search(query):
    global Querylist
    global visit
    global TOPK
    Querylist = token(query)
#     print(Querylist)
    for TERM in Querylist:
#         print('now it turns to %s'%TERM)
        if TERM in postings.keys():
#             print('%s in postings.keys()'%TERM)
            for doc in postings[TERM]:
                if doc[0] not in visit.keys():
                    visit[doc[0]] = Search_document(doc[0])

    print(sorted(visit.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)[:TOPK])
    

In [41]:
get_postings()

In [56]:
Search('Manufacturers of industrial controllers')

[('317377292047376387', 0.6172668771549408), ('28977078074343425', 0.5414310555540102), ('30996863431745536', 0.5079336196604043), ('302154195702800384', 0.476187768431629), ('625192875345768448', 0.47202761194201354), ('29980635686772738', 0.4223404948954858), ('30172846525251585', 0.40122347015071147), ('34780760116305920', 0.4010002260476876), ('30172275693068288', 0.3821175906197252), ('30147216362446848', 0.3821175906197252)]


In [39]:
print([postings.keys()][0])

dict_keys([('mariah', 1), ('people', 1), ('house', 2), ('may', 1), ('kill', 1), ('arizona-style', 1), ('immigration', 1), ('bill', 1), ('rep', 1), ('rick', 1), ('rand', 1), ('say', 1), ('the', 2), ('be', 1), ('unlikely', 1), ('to', 1), ('pas', 1), ('arus', 1), ('http', 1), ('tinyurl.com/4jrjcdz', 1), ('servando', 1), ('mourner', 1), ('recall', 1), ('sarge', 1), ('shriver', 2), ("'", 1), ('charity', 1), ('idealism', 1), ('n', 1), ('ap', 2), ('r', 1), ('sargent', 1), ('wa', 1), ('alway', 1), ('an', 1), ('optimist', 1), ('pio', 1), ('bit.ly/gqmcdg', 1), ('heide', 1), ('eversoll', 1), ('bas', 1), ('fish', 1), ('technique', 1), ('2', 1), ('fantastic', 1), ('tip', 1), ('improve', 1), ('ymy', 1), ('cast', 1), ('skill', 1), ('ailsa', 1), ('hang', 1), ('financial', 2), ('aid', 2), ('proper', 1), ('method', 1), ('of', 1), ('get', 1), ('for', 1), ('education', 1), ('ping.fm/bk0r3', 1), ('applying-for-financial-aid', 1), ('financial-aid-essay', 1), ('brothy', 1), ('supreme', 1), ('court', 1), ('na